In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras


In [ ]:
mnist=tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test)=mnist.load_data()

In [ ]:
#normalizes color values to 0-1
x_train=tf.keras.utils.normalize(x_train, axis=1)
x_test=tf.keras.utils.normalize(x_test, axis=1)

In [ ]:
x_train=x_train.reshape(-1,28*28)   #flatten images
x_test=x_test.reshape(-1,28*28)

model = keras.Sequential([
    keras.Input(shape=(28*28,)), # Feature layer
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train,epochs=3)

model.save('handwritten.keras')




In [ ]:
loss, accuracy = model.evaluate(x_test,y_test)
print(loss)
print(accuracy)

In [ ]:
from google.colab import files
import zipfile
import os

uploaded = files.upload()

for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

    # This makes a directory and takes a zip file
    extract_dir = filename.replace('.zip', '')
    os.makedirs(extract_dir, exist_ok=True)

    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

print(f'Contents extracted to: {extract_dir}')

In [ ]:
extracted_files = os.listdir(extract_dir)
print("Contents of the extracted directory:")
for file in extracted_files:
    print(file)

In [ ]:

for filename in extracted_files:
    if filename.endswith(('.png', '.jpg', '.jpeg')):  #use whatever depending on files
        try:
            img_path = os.path.join(extract_dir, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale


            img = cv2.resize(img, (28, 28))

            # inverts colors, had a problem with them being flipped
            img = cv2.bitwise_not(img)

            # Normalize and flatten the image
            img = img.astype('float32') / 255.0
            img = img.flatten()
            img = img.reshape(1, 784)

            prediction = model.predict(img)
            print(f"This digit is probably a {np.argmax(prediction)}")

            plt.imshow(img.reshape(28, 28), cmap=plt.cm.binary)
            plt.title(f"Predicted: {np.argmax(prediction)}")
            plt.show()
        except Exception as e:
            print(f'Error processing file {filename}: {e}')
